<a href="https://colab.research.google.com/github/mosesyhc/de300-wn2024-notes/blob/main/examples/ex-pipeline-titanic-full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Retrieving Java, Spark, and `findspark` in Python

In [ ]:
!pip install -q findspark
!pip install -q seaborn

In [ ]:
!pip install pyspark

# `titanic` Dataset

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic', data_home='dataset/', cache=True)

In [ ]:
titanic = spark.read.csv(
    "dataset/titanic.csv", inferSchema=True, header=True
)

# PySpark setup

In [ ]:
from pyspark.sql import SparkSession

# initiate spark session
spark = (
    SparkSession.builder
    .appName("ML Pipeline")
    .getOrCreate()
)


In [ ]:
titanic.printSchema()

In [ ]:
titanic.describe().show()

In [ ]:
# clean up duplicate columns
titanic = titanic.drop('deck', 'embark_town', 'alive', 'class')

In [ ]:
titanic.describe().show()

In [ ]:
titanic.select('embarked').distinct().show()

In [ ]:
# continuous variables
CONTINUOUS_COLUMNS = [
    'pclass',
    'age',
    'sibsp',
    'parch',
    'fare',
]

# binary text variables
BINARY_COLUMNS = [
    'sex'
]

# categorical variables
CATEGORICAL_COLUMNS = [
    'embarked',
    'who'
]

# boolean variables
BOOLEAN_COLUMNS = [
    'adult_male',
    'alone'
]

In [ ]:
# transform boolean columns
import pyspark.sql.types as T
for x in BOOLEAN_COLUMNS:
  titanic = titanic.withColumn(x, titanic[x].cast(T.IntegerType()))

# Estimator

In [ ]:
import pyspark.ml.feature as MF

# transform sex column
sex_indexer = MF.StringIndexer(inputCol='sex',
                               outputCol='sex_index')

# transform who column
who_indexer = MF.StringIndexer(inputCol='who',
                               outputCol='who_index')
who_encoder = MF.OneHotEncoder(inputCol='who_index',
                               outputCol='who_vec')

# transform embarked column
embarked_indexer = MF.StringIndexer(inputCol='embarked',
                                  outputCol='embarked_index',
                                  handleInvalid='skip')
embarked_encoder = MF.OneHotEncoder(inputCol='embarked_index_impute',
                                  outputCol='embarked_vec')

In [ ]:
# age imputation
age_imputer  = MF.Imputer(strategy='median',
                          inputCol='age',
                          outputCol='age')

# embarked imputation
embarked_imputer = MF.Imputer(strategy='mode',
                              inputCol='embarked_index',
                              outputCol='embarked_index_impute')

# Transformer

In [ ]:
# collecting all predictors / features
assembler = MF.VectorAssembler(inputCols=CONTINUOUS_COLUMNS
                               + ['sex_index']
                               + ['embarked_vec']
                               + ['who_vec'],
                               outputCol='features')


# Pre-training pipeline



In [ ]:
from pyspark.ml import Pipeline

titanic_pipeline = Pipeline(
    stages=[
        age_imputer,
        sex_indexer,
        who_indexer,
        who_encoder,
        embarked_indexer,
        embarked_imputer,
        embarked_encoder,
        assembler
    ]
)

In [ ]:
assembled_titanic = titanic_pipeline.fit(titanic).transform(titanic)

In [ ]:
assembled_titanic.show(10)

# Logistic Regression model

In [ ]:
# logistic regression
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',labelCol='survived')

In [ ]:
lr_titanic = lr.fit(assembled_titanic)

In [ ]:
titanic_pred = lr_titanic.transform(assembled_titanic)

In [ ]:
titanic_pred.show(5)

# Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator as bcEval

binary_eval = bcEval(rawPredictionCol='prediction',
                     labelCol='survived')

In [ ]:
titanic_eval = binary_eval.evaluate(titanic_pred)

In [ ]:
print(titanic_eval)

# Understanding the model via extraction of coefficients

In [ ]:
lr_titanic.intercept

In [ ]:
lr_titanic.coefficients

In [ ]:
print(CONTINUOUS_COLUMNS
+ ['sex_index']
+ ['embarked_vec']
+ ['who_vec']
)